In [ ]:
import os
import path_file
os.chdir(os.path.dirname(os.path.dirname(path_file.__file__)))
os.getcwd()

In [88]:
import sys
import cantera as ct
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt

import phases
import feedstock as fs
import fuel as fu
import energy as en
import gasifier as gs

In [89]:
birchUltimate = [82.5, 3.6, 13.9, 0, 0, 0]
birchAsh = 0.035
birchMoist1 = 0.036 / (1 - 0.036)
birchMoist2 = 0.044 / (1 - 0.044)
birchMoist3 = 0.026 / (1 - 0.026)
birchHHV = 0.3491*birchUltimate[0] + 1.1783*birchUltimate[1] + 0.1005*birchUltimate[4] - 0.1034*birchUltimate[2] - 0.0151*birchUltimate[3] - 0.0211*birchAsh*100
print('HHV =', birchHHV, 'MJ/kg')

birch1 = fs.create_fuel_stream(0.73*(1-0.036), birchUltimate, birchAsh, birchMoist1, 20)
birch2 = fs.create_fuel_stream(1, birchUltimate, birchAsh, birchMoist2, birchHHV)
birch3 = fs.create_fuel_stream(1, birchUltimate, birchAsh, birchMoist3, birchHHV)
print(birch1.get_mass())

HHV = 31.53152 MJ/kg
0.7299999999999999


In [90]:
def kmoles(Nm3):
    return 101325 * Nm3 / (8.31446262 * 273.15) / 1000

In [91]:
stoichAir = fs.create_air_from_ER(birch1, 1.0)
stoichAirMass = stoichAir.get_mass()
air = fs.create_air_from_ER(birch1, 0.567)
airKmol = kmoles(2.43)
airMoist = 0.01
airMoles = air.species_moles
airMoles[phases.indices['N2']] = (1-airMoist) * airKmol * 0.78
airMoles[phases.indices['O2']] = (1-airMoist) * airKmol * 0.21
airMoles[phases.indices['Ar']] = (1-airMoist) * airKmol * 0.01
airMoles[phases.indices['H2O']] = airMoist * airKmol
air.species_moles = airMoles
print(sum(air.species_moles))

0.10841453115121244


In [92]:
# outlet = gs.gasify_isot(birch1, air, T=812+273.15, P=101325, charFormation=0.014, directMethaneConv=0.0)
outlet = gs.gasify_nonisot(birch1, air, T0=23+273.18, P=101325, heatLossFraction=0.0, charFormation=0.0, directMethaneConv=0.0)
print(outlet.T - 273.15)
species = ['H2', 'O2', 'N2', 'Ar', 'CO', 'CH4', 'CO2', 'H2O']
soma = 0
for i, sp in enumerate(species):
    frac = 100*outlet.get_syngas_fraction(sp, water=True, nitrogen=True)
    print(f'{sp}: {frac}')
    soma += frac
print(soma)
print(outlet.get_syngas_amount(basis='vol', water=True, nitrogen=True))
print('CC:', outlet.species_moles[phases.indices['C(gr)']] / birch1.species_moles[phases.indices['C(gr)']])
print('C:', outlet.species_moles[phases.indices['C(gr)']] * phases.Mw['C(gr)'] * 1000)

510.35946349477865
H2: 5.087213962327083
O2: 7.440376537804492e-26
N2: 67.5397412618868
Ar: 0.8659155457028334
CO: 3.1371241978690496
CH4: 0.44930497589573837
CO2: 17.077914475333557
H2O: 5.839443918794927
99.99665833781
2.7782155100507366
CC: 0.4501458482285209
C: 252.19379958657774
